# Funzioni d'utilità

In [1]:
import os
import shutil
import json
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
def remove_and_make_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
        os.mkdir(path)
    else:
        os.mkdir(path)

In [3]:
def reformat_json(path):
    with open(path, "wr") as f:
        tmp = json.loads(f.read())
        json_obj = json.dumps(tmp, indent=4)
        f.write(json_obj)
        f.close()

In [4]:
def jaccard(set1, set2):
    intersection_size = len(set1 & set2)
    union_size = len(set1 | set2)
    if union_size > 0:
        return intersection_size / union_size
    else:
        return 0

In [38]:
from itertools import combinations
import pandas as pd
from Levenshtein import distance

In [49]:
def extract_schema_paolo(directory, name, final_schema_directory, t_jaccard, t_edit, format, head):

    result_jaccard = {}
    for filename in os.listdir(directory):

        print(filename)
        file = os.path.join(directory, filename)
        df = pd.read_csv(file, encoding="utf-8", low_memory=False)
        df = df.head(head)
        
        columns = list(df.columns)
        n_cols = len(columns)
        combs = combinations(range(n_cols), 2)
        print("Numbero di confronti: ", n_cols * (n_cols-1) // 2)
        k = 1
                
        for c1, c2 in combs:
            
            cc1 = columns[c1]
            cc2 = columns[c2]
            
            tmp1 = cc1.split("-")[-1]
            tmp2 = cc2.split("-")[-1]
            relative_edit_distance = distance(tmp1, tmp2) / max(len(tmp1), len(tmp2))
            print("EDIT", k, cc1, cc2, relative_edit_distance)
            if relative_edit_distance < t_edit:
                result_jaccard.setdefault(cc1, set()).add(cc2)
                result_jaccard.setdefault(cc2, set()).add(cc1)
                
            else:
                set1 = set(df.iloc[:, c1])
                set2 = set(df.iloc[:, c2])

                s = jaccard(set1, set2)
                print("JACCARD", k, cc1, cc2, s)
                if s > t_jaccard:
                    result_jaccard.setdefault(cc1, set()).add(cc2)
                    result_jaccard.setdefault(cc2, set()).add(cc1)
            
            k += 1
        
        del df

    s = set()
    for main_token, tokens in result_jaccard.items():
        tmp = set()
        tmp.add(main_token)
        tmp.update(tokens)
        s.add(frozenset(tmp))

    dict_synonyms = {}
    for tokens in s:
        tokens = set(tokens)
        tmp = list(tokens)[0].split("-")[-1]
        if tmp not in dict_synonyms.keys() and tokens != None:
            dict_synonyms[tmp] = tokens
        elif dict_synonyms[tmp] != None:
            dict_synonyms[tmp] = dict_synonyms[tmp].update(tokens)
            
    dict_synonyms = {k:list(v) for k,v in dict_synonyms.items() if v!=None}
            
    json_obj = json.dumps(dict_synonyms, indent=4)
    f = open(final_schema_directory + name + format, "w")
    f.write(json_obj)
    f.close()

    print("\n\n")

# Schemi mediati cluster

In [7]:
from task_1 import make_cluster_columns_cluster
#from task_2 import schema_cluster
from task_4 import make_schema_cluster

In [8]:
clusters_datasets_base_path = "..\\..\\Dataset\\ClusterParsed\\"
clusters_info_path = "..\\DatasetSchemaMatch\\"
clusters_filename_synonym = "dic_pre_val.txt"

clusters = ".\\clusters\\"
clusters_columns = ".\\clusters\\columns\\"
clusters_dictionary = ".\\clusters\\dictionary\\"
clusters_final_dictionary = ".\\clusters\\final_dictionary\\"
clusters_schema = ".\\clusters\\schema\\"

In [ ]:
#remove_and_make_dir(clusters)

### TASK 1

In [ ]:
print("\nMake cluster columns\n")
remove_and_make_dir(clusters_columns)
make_cluster_columns_cluster(clusters_datasets_base_path, clusters_info_path, clusters_filename_synonym, clusters_columns)

### TASK 2

In [51]:
def schema_cluster(directory, final_schema_directory, t_jaccard, t_edit, format, head):
    for dirname in os.listdir(directory):
        print(dirname)
        cluster = os.path.join(directory, dirname)
        extract_schema_paolo(cluster, dirname, final_schema_directory, t_jaccard, t_edit, format, head)

In [52]:
print("\nExtract schema\n")
remove_and_make_dir(clusters_dictionary)
start = time.time()
schema_cluster(clusters_columns, clusters_dictionary, 0.1, 0.0, ".txt", 10000)
end = time.time()
print("\nTempo di esecuzione: "+str(end-start)+" (s)"+"\n")


Extract schema

ambitiobox
foundation_year.csv
Numbero di confronti:  0
industry.csv
Numbero di confronti:  3
EDIT 1 06-MarScoToc-industry 06-MarScoToc-headquarter 0.7272727272727273
JACCARD 1 06-MarScoToc-industry 06-MarScoToc-headquarter 0.0005173305742369374
EDIT 2 06-MarScoToc-industry 06-MarScoToc-name 0.875
JACCARD 2 06-MarScoToc-industry 06-MarScoToc-name 0.00021748586341887777
EDIT 3 06-MarScoToc-headquarter 06-MarScoToc-name 0.8181818181818182
JACCARD 3 06-MarScoToc-headquarter 06-MarScoToc-name 0.0
ownership.csv
Numbero di confronti:  0



ariregister
address.csv
Numbero di confronti:  0
capital.csv
Numbero di confronti:  0
id.csv
Numbero di confronti:  0
legal_form.csv
Numbero di confronti:  0
registration_date.csv
Numbero di confronti:  3
EDIT 1 09-wissel-registration_date 09-wissel-deletion_date 0.4117647058823529
JACCARD 1 09-wissel-registration_date 09-wissel-deletion_date 0.05737704918032787
EDIT 2 09-wissel-registration_date 09-wissel-code 0.8235294117647058
JACCARD 2

JACCARD 11 06-MarScoToc-master_cap 02-GioPonSpiz-pricecap 0.2985430463576159
EDIT 12 06-MarScoToc-master_cap 03-gren-market_capitalization_usd 0.72
JACCARD 12 06-MarScoToc-master_cap 03-gren-market_capitalization_usd 0.23434343434343435
EDIT 13 06-MarScoToc-master_cap 09-wissel-revenue 0.9
JACCARD 13 06-MarScoToc-master_cap 09-wissel-revenue 0.2533449680046539
EDIT 14 06-MarScoToc-master_cap 01-DDD-marketcap 0.4
JACCARD 14 06-MarScoToc-master_cap 01-DDD-marketcap 0.2992042440318302
EDIT 15 06-MarScoToc-master_cap 04-iGMM-marketcap 0.4
JACCARD 15 06-MarScoToc-master_cap 04-iGMM-marketcap 0.29668271803103263
EDIT 16 06-MarScoToc-master_cap 05-MalPatSaj-marketcap 0.4
JACCARD 16 06-MarScoToc-master_cap 05-MalPatSaj-marketcap 0.30321175830157865
EDIT 17 06-MarScoToc-master_cap 09-wissel-marketcap 0.4
JACCARD 17 06-MarScoToc-master_cap 09-wissel-marketcap 0.2890258215962441
EDIT 18 09-wissel-earnings 02-GioPonSpiz-pricecap 1.0
JACCARD 18 09-wissel-earnings 02-GioPonSpiz-pricecap 0.1140529531

JACCARD 13 01-DDD-name 09-wissel-name 0.7320143884892086
EDIT 14 02-GioPonSpiz-name 03-gren-name 0.0
JACCARD 14 02-GioPonSpiz-name 03-gren-name 0.9898835920177383
EDIT 15 02-GioPonSpiz-name 04-iGMM-name 0.0
JACCARD 15 02-GioPonSpiz-name 04-iGMM-name 0.9786762669620603
EDIT 16 02-GioPonSpiz-name 05-MalPatSaj-name 0.0
JACCARD 16 02-GioPonSpiz-name 05-MalPatSaj-name 0.9605026929982047
EDIT 17 02-GioPonSpiz-name 06-MarScoToc-name 0.0
JACCARD 17 02-GioPonSpiz-name 06-MarScoToc-name 0.8618166758696852
EDIT 18 02-GioPonSpiz-name 09-wissel-name 0.0
JACCARD 18 02-GioPonSpiz-name 09-wissel-name 0.7201430930104568
EDIT 19 03-gren-name 04-iGMM-name 0.0
JACCARD 19 03-gren-name 04-iGMM-name 0.988698200083717
EDIT 20 03-gren-name 05-MalPatSaj-name 0.0
JACCARD 20 03-gren-name 05-MalPatSaj-name 0.969810795770729
EDIT 21 03-gren-name 06-MarScoToc-name 0.0
JACCARD 21 03-gren-name 06-MarScoToc-name 0.853382940609067
EDIT 22 03-gren-name 09-wissel-name 0.0
JACCARD 22 03-gren-name 09-wissel-name 0.723113077

JACCARD 71 00-avengers-share_price 00-avengers-rank 0.0
EDIT 72 00-avengers-share_price 01-DDD-rank 0.9090909090909091
JACCARD 72 00-avengers-share_price 01-DDD-rank 0.0
EDIT 73 00-avengers-share_price 04-iGMM-rank 0.9090909090909091
JACCARD 73 00-avengers-share_price 04-iGMM-rank 0.0
EDIT 74 00-avengers-share_price 06-MarScoToc-rank 0.9090909090909091
JACCARD 74 00-avengers-share_price 06-MarScoToc-rank 0.0001015228426395939
EDIT 75 00-avengers-share_price 04-iGMM-change_1day 0.7272727272727273
JACCARD 75 00-avengers-share_price 04-iGMM-change_1day 0.00020470829068577277
EDIT 76 01-DDD-share_price 06-MarScoToc-share_price 0.0
JACCARD 76 01-DDD-share_price 06-MarScoToc-share_price 0.293290539507871
EDIT 77 01-DDD-share_price 09-wissel-share_price 0.0
JACCARD 77 01-DDD-share_price 09-wissel-share_price 0.2728862047043865
EDIT 78 01-DDD-share_price 01-DDD-change1y 0.8181818181818182
JACCARD 78 01-DDD-share_price 01-DDD-change1y 0.0
EDIT 79 01-DDD-share_price 02-GioPonSpiz-price 0.5454545

JACCARD 149 01-DDD-rank 06-MarScoToc-rank 0.0
EDIT 150 01-DDD-rank 04-iGMM-change_1day 0.8181818181818182
JACCARD 150 01-DDD-rank 04-iGMM-change_1day 0.0
EDIT 151 04-iGMM-rank 06-MarScoToc-rank 0.0
JACCARD 151 04-iGMM-rank 06-MarScoToc-rank 0.0
EDIT 152 04-iGMM-rank 04-iGMM-change_1day 0.8181818181818182
JACCARD 152 04-iGMM-rank 04-iGMM-change_1day 0.0
EDIT 153 06-MarScoToc-rank 04-iGMM-change_1day 0.8181818181818182
JACCARD 153 06-MarScoToc-rank 04-iGMM-change_1day 0.0001313370107696349
shares.csv
Numbero di confronti:  0
url.csv
Numbero di confronti:  0



disfold
categories.csv
Numbero di confronti:  0
ceo.csv
Numbero di confronti:  6
EDIT 1 00-avengers-ceo 04-iGMM-ceo 0.0
JACCARD 1 00-avengers-ceo 04-iGMM-ceo 0.40467937608318894
EDIT 2 00-avengers-ceo 08-slytherin-ceo 0.0
JACCARD 2 00-avengers-ceo 08-slytherin-ceo 0.025380710659898477
EDIT 3 00-avengers-ceo 10-DeBiGa-ceo 0.0
JACCARD 3 00-avengers-ceo 10-DeBiGa-ceo 0.0006172839506172839
EDIT 4 04-iGMM-ceo 08-slytherin-ceo 0.0
JACCAR

...

JACCARD 1 03-gren-revenue_2017_eu 01-DDD-revenue_2020_euro 0.0
EDIT 2 03-gren-revenue_2017_eu 03-gren-revenue_2020_eu 0.13333333333333333
JACCARD 2 03-gren-revenue_2017_eu 03-gren-revenue_2020_eu 0.0
EDIT 3 03-gren-revenue_2017_eu 07-silvestri-revenue_2020 0.3333333333333333
JACCARD 3 03-gren-revenue_2017_eu 07-silvestri-revenue_2020 0.0
EDIT 4 03-gren-revenue_2017_eu 08-slytherin-revenue 0.5333333333333333
JACCARD 4 03-gren-revenue_2017_eu 08-slytherin-revenue 0.0
EDIT 5 01-DDD-revenue_2020_euro 03-gren-revenue_2020_eu 0.11764705882352941
JACCARD 5 01-DDD-revenue_2020_euro 03-gren-revenue_2020_eu 0.9974126778783958
EDIT 6 01-DDD-revenue_2020_euro 07-silvestri-revenue_2020 0.29411764705882354
JACCARD 6 01-DDD-revenue_2020_euro 07-silvestri-revenue_2020 1.0
EDIT 7 01-DDD-revenue_2020_euro 08-slytherin-revenue 0.5882352941176471
JACCARD 7 01-DDD-revenue_2020_euro 08-slytherin-revenue 0.3232704402515723
EDIT 8 03-gren-revenue_2020_eu 07-silvestri-revenue_2020 0.2
JACCARD 8 03-gren-revenue

JACCARD 3 03-gren-annual_net_income_usd 04-iGMM-annual_net_income_in_usd 0.0
EDIT 4 03-gren-annual_net_income_usd 03-gren-annual_revenue_usd 0.3333333333333333
JACCARD 4 03-gren-annual_net_income_usd 03-gren-annual_revenue_usd 0.37411764705882355
EDIT 5 04-iGMM-market_value__jan_1st_2020 00-avengers-annual_net_income_in_usd 0.8461538461538461
JACCARD 5 04-iGMM-market_value__jan_1st_2020 00-avengers-annual_net_income_in_usd 0.0
EDIT 6 04-iGMM-market_value__jan_1st_2020 04-iGMM-annual_net_income_in_usd 0.8461538461538461
JACCARD 6 04-iGMM-market_value__jan_1st_2020 04-iGMM-annual_net_income_in_usd 0.0
EDIT 7 04-iGMM-market_value__jan_1st_2020 03-gren-annual_revenue_usd 0.8076923076923077
JACCARD 7 04-iGMM-market_value__jan_1st_2020 03-gren-annual_revenue_usd 0.021789231091556675
EDIT 8 00-avengers-annual_net_income_in_usd 04-iGMM-annual_net_income_in_usd 0.0
JACCARD 8 00-avengers-annual_net_income_in_usd 04-iGMM-annual_net_income_in_usd 0.39475235849056606
EDIT 9 00-avengers-annual_net_i

JACCARD 1 08-slytherin-industry 00-avengers-company_business 0.966078697421981
EDIT 2 08-slytherin-industry 04-iGMM-company_business 0.8125
JACCARD 2 08-slytherin-industry 04-iGMM-company_business 0.22830859293330547
EDIT 3 00-avengers-company_business 04-iGMM-company_business 0.0
JACCARD 3 00-avengers-company_business 04-iGMM-company_business 0.21371051001445385
link.csv
Numbero di confronti:  0
market_capitalization_2022.csv
Numbero di confronti:  36
EDIT 1 07-silvestri-market_value_jan_2020 03-gren-market_capitalization_2022 0.46153846153846156
JACCARD 1 07-silvestri-market_value_jan_2020 03-gren-market_capitalization_2022 0.008797653958944282
EDIT 2 07-silvestri-market_value_jan_2020 07-silvestri-revenue_2022 0.5714285714285714
JACCARD 2 07-silvestri-market_value_jan_2020 07-silvestri-revenue_2022 0.0006090133982947625
EDIT 3 07-silvestri-market_value_jan_2020 07-silvestri-market_value_jan_2022 0.047619047619047616
JACCARD 3 07-silvestri-market_value_jan_2020 07-silvestri-market_va

JACCARD 11 03-gren-market_capitalization_2022 07-silvestri-market_value_jan_2022 0.010061668289516391
EDIT 12 03-gren-market_capitalization_2022 04-iGMM-market_value__jan_1st_2020 0.5384615384615384
JACCARD 12 03-gren-market_capitalization_2022 04-iGMM-market_value__jan_1st_2020 0.01928312159709619
EDIT 13 03-gren-market_capitalization_2022 04-iGMM-market_value__jan_07_2022 0.46153846153846156
JACCARD 13 03-gren-market_capitalization_2022 04-iGMM-market_value__jan_07_2022 0.022910757145974254
EDIT 14 03-gren-market_capitalization_2022 08-slytherin-revenue 0.8846153846153846
JACCARD 14 03-gren-market_capitalization_2022 08-slytherin-revenue 0.0
EDIT 15 03-gren-market_capitalization_2022 02-GioPonSpiz-marketvalue 0.6923076923076923
JACCARD 15 03-gren-market_capitalization_2022 02-GioPonSpiz-marketvalue 0.021432945499081445
EDIT 16 03-gren-market_capitalization_2022 02-GioPonSpiz-marketcap 0.6538461538461539
JACCARD 16 03-gren-market_capitalization_2022 02-GioPonSpiz-marketcap 0.023495465

EDIT 1 00-avengers-total_liabilities_in_usd 00-avengers-total_equity_in_usd 0.375
JACCARD 1 00-avengers-total_liabilities_in_usd 00-avengers-total_equity_in_usd 0.25324506749740394
EDIT 2 00-avengers-total_liabilities_in_usd 00-avengers-total_assets_in_usd 0.3333333333333333
JACCARD 2 00-avengers-total_liabilities_in_usd 00-avengers-total_assets_in_usd 0.19232864770622207
EDIT 3 00-avengers-total_liabilities_in_usd 00-avengers-annual_revenue_in_usd 0.5833333333333334
JACCARD 3 00-avengers-total_liabilities_in_usd 00-avengers-annual_revenue_in_usd 0.24344096871846618
EDIT 4 00-avengers-total_liabilities_in_usd 04-iGMM-annual_revenue_in_usd 0.5833333333333334
JACCARD 4 00-avengers-total_liabilities_in_usd 04-iGMM-annual_revenue_in_usd 0.09234009533311163
EDIT 5 00-avengers-total_equity_in_usd 00-avengers-total_assets_in_usd 0.2631578947368421
JACCARD 5 00-avengers-total_equity_in_usd 00-avengers-total_assets_in_usd 0.22568229531140657
EDIT 6 00-avengers-total_equity_in_usd 00-avengers-an

### TASK 4

In [ ]:
print("\nMake schemas\n")
remove_and_make_dir(clusters_schema)
make_schema_cluster(clusters_datasets_base_path, clusters_final_dictionary, clusters_schema, ".txt")

# Schema mediato finale

In [ ]:
from task_1 import make_cluster_columns_final
#from task_2 import schema_final
from task_4 import make_schema_final

In [45]:
final_datasets_base_path = ".\\clusters\\schema\\"
final_info_path = ".\\files_matching\\files_vari"
final_filename_synonym = "dic_pre_val.txt"

final_schema = ".\\final_schema\\"
final_schema_columns = ".\\final_schema\\columns\\"
final_schema_dictionary = ".\\final_schema\\dictionary\\"
final_schema_final_dictionary = ".\\final_schema\\final_dictionary\\"
final_schema_schema = ".\\final_schema\\schema\\"

In [ ]:
#remove_and_make_dir(final_schema)

### TASK 1

In [ ]:
print("\nMake cluster columns\n")
remove_and_make_dir(final_schema_columns)
make_cluster_columns_final(final_datasets_base_path, final_info_path, final_filename_synonym, final_schema_columns)

### TASK 2

In [53]:
def schema_final(directory, final_schema_directory, t_jaccard, t_edit, format, head):
    extract_schema_paolo(directory, "finalschema", final_schema_directory, t_jaccard, t_edit, format, head)

In [54]:
print("\nExtract schema\n")
remove_and_make_dir(final_schema_dictionary)
start = time.time()
schema_final(final_schema_columns, final_schema_dictionary, 0.1, 0.5, ".txt", 5000)
end = time.time()
print("\nTempo di esecuzione: "+str(end-start)+" (s)"+"\n")


Extract schema

address.csv
Numbero di confronti:  6
EDIT 1 infoclipper-address_name globaldata-address 0.4166666666666667
EDIT 2 infoclipper-address_name hithorizons-address 0.4166666666666667
EDIT 3 infoclipper-address_name govuk-office_address 0.8571428571428571
JACCARD 3 infoclipper-address_name govuk-office_address 0.00024789291026276647
EDIT 4 globaldata-address hithorizons-address 0.0
EDIT 5 globaldata-address govuk-office_address 0.5
JACCARD 5 globaldata-address govuk-office_address 0.0
EDIT 6 hithorizons-address govuk-office_address 0.5
JACCARD 6 hithorizons-address govuk-office_address 0.0005431830526887561
categories.csv
Numbero di confronti:  136
EDIT 1 wikipedia-notes disfold-sector 0.8333333333333334
JACCARD 1 wikipedia-notes disfold-sector 0.004120879120879121
EDIT 2 wikipedia-notes ft-sector 0.8333333333333334
JACCARD 2 wikipedia-notes ft-sector 0.01084010840108401
EDIT 3 wikipedia-notes valuetoday-sector 0.8333333333333334
JACCARD 3 wikipedia-notes valuetoday-sector 0

JACCARD 90 ariregister-industry companiesmarketcap-categories 0.0
EDIT 91 ariregister-industry disfold-categories 0.9
JACCARD 91 ariregister-industry disfold-categories 0.0
EDIT 92 cbinsights-industry disfold-industry 0.0
EDIT 93 cbinsights-industry forbes-industry 0.0
EDIT 94 cbinsights-industry globaldata-industry 0.0
EDIT 95 cbinsights-industry hithorizons-industry 0.0
EDIT 96 cbinsights-industry teamblind-industry 0.0
EDIT 97 cbinsights-industry valuetoday-industry 0.0
EDIT 98 cbinsights-industry wikipedia-industry 0.0
EDIT 99 cbinsights-industry companiesmarketcap-categories 0.9
JACCARD 99 cbinsights-industry companiesmarketcap-categories 0.001081081081081081
EDIT 100 cbinsights-industry disfold-categories 0.9
JACCARD 100 cbinsights-industry disfold-categories 0.002036659877800407
EDIT 101 disfold-industry forbes-industry 0.0
EDIT 102 disfold-industry globaldata-industry 0.0
EDIT 103 disfold-industry hithorizons-industry 0.0
EDIT 104 disfold-industry teamblind-industry 0.0
EDIT 10

JACCARD 30 globaldata-headquarters ariregister-ownership 0.0
EDIT 31 teamblind-headquarters wikipedia-headquarters 0.0
EDIT 32 teamblind-headquarters disfold-headquarters_continent 0.45454545454545453
EDIT 33 teamblind-headquarters valuetoday-headquarters_continent 0.45454545454545453
EDIT 34 teamblind-headquarters disfold-headquarters_country 0.4
EDIT 35 teamblind-headquarters cbinsights-city 0.9166666666666666
JACCARD 35 teamblind-headquarters cbinsights-city 0.002932551319648094
EDIT 36 teamblind-headquarters infoclipper-city 0.9166666666666666
JACCARD 36 teamblind-headquarters infoclipper-city 0.0006954102920723226
EDIT 37 teamblind-headquarters ambitiobox-ownership 0.9166666666666666
JACCARD 37 teamblind-headquarters ambitiobox-ownership 0.0023923444976076554
EDIT 38 teamblind-headquarters ariregister-ownership 0.9166666666666666
JACCARD 38 teamblind-headquarters ariregister-ownership 0.0
EDIT 39 wikipedia-headquarters disfold-headquarters_continent 0.45454545454545453
EDIT 40 wik

EDIT 77 infoclipper-country disfold-headquarters_continent 0.8636363636363636
JACCARD 77 infoclipper-country disfold-headquarters_continent 0.010752688172043012
EDIT 78 infoclipper-country valuetoday-headquarters_continent 0.8636363636363636
JACCARD 78 infoclipper-country valuetoday-headquarters_continent 0.1
EDIT 79 infoclipper-country disfold-headquarters_country 0.65
JACCARD 79 infoclipper-country disfold-headquarters_country 0.002531645569620253
EDIT 80 infoclipper-country infoclipper-state 0.8571428571428571
JACCARD 80 infoclipper-country infoclipper-state 0.08163265306122448
EDIT 81 infoclipper-country disfold-headquarters 0.75
JACCARD 81 infoclipper-country disfold-headquarters 0.0022935779816513763
EDIT 82 infoclipper-country globaldata-headquarters 0.75
JACCARD 82 infoclipper-country globaldata-headquarters 0.023529411764705882
EDIT 83 infoclipper-country teamblind-headquarters 0.75
JACCARD 83 infoclipper-country teamblind-headquarters 0.0024691358024691358
EDIT 84 infoclipper

JACCARD 55 companiesmarketcap-rank valuetoday-employees_number 0.0
EDIT 56 companiesmarketcap-rank globaldata-number_of_employees 0.9473684210526315
JACCARD 56 companiesmarketcap-rank globaldata-number_of_employees 0.0
EDIT 57 disfold-rank valuetoday-rank 0.0
EDIT 58 disfold-rank ft-revenue_2017_eu 0.8666666666666667
JACCARD 58 disfold-rank ft-revenue_2017_eu 0.00033344448149383126
EDIT 59 disfold-rank valuetoday-employees_number 0.9375
JACCARD 59 disfold-rank valuetoday-employees_number 0.0
EDIT 60 disfold-rank globaldata-number_of_employees 0.9473684210526315
JACCARD 60 disfold-rank globaldata-number_of_employees 0.04165776543473617
EDIT 61 valuetoday-rank ft-revenue_2017_eu 0.8666666666666667
JACCARD 61 valuetoday-rank ft-revenue_2017_eu 0.0005002501250625312
EDIT 62 valuetoday-rank valuetoday-employees_number 0.9375
JACCARD 62 valuetoday-rank valuetoday-employees_number 0.0
EDIT 63 valuetoday-rank globaldata-number_of_employees 0.9473684210526315
JACCARD 63 valuetoday-rank globalda

...

JACCARD 56 ariregister-industry ambitiobox-name 0.0
EDIT 57 ariregister-industry ariregister-name 0.875
JACCARD 57 ariregister-industry ariregister-name 0.0
EDIT 58 ariregister-industry cbinsights-name 0.875
JACCARD 58 ariregister-industry cbinsights-name 0.0
EDIT 59 ariregister-industry companiesmarketcap-name 0.875
JACCARD 59 ariregister-industry companiesmarketcap-name 0.0
EDIT 60 ariregister-industry disfold-name 0.875
JACCARD 60 ariregister-industry disfold-name 0.0
EDIT 61 ariregister-industry forbes-name 0.875
JACCARD 61 ariregister-industry forbes-name 0.0
EDIT 62 ariregister-industry ft-name 0.875
JACCARD 62 ariregister-industry ft-name 0.0
EDIT 63 ariregister-industry globaldata-name 0.875
JACCARD 63 ariregister-industry globaldata-name 0.0
EDIT 64 ariregister-industry govuk-name 0.875
JACCARD 64 ariregister-industry govuk-name 0.0
EDIT 65 ariregister-industry hithorizons-name 0.875
JACCARD 65 ariregister-industry hithorizons-name 0.0
EDIT 66 ariregister-industry infoclipper-

EDIT 154 hithorizons-industry companiesmarketcap-name 0.875
JACCARD 154 hithorizons-industry companiesmarketcap-name 0.0
EDIT 155 hithorizons-industry disfold-name 0.875
JACCARD 155 hithorizons-industry disfold-name 0.0
EDIT 156 hithorizons-industry forbes-name 0.875
JACCARD 156 hithorizons-industry forbes-name 0.0
EDIT 157 hithorizons-industry ft-name 0.875
JACCARD 157 hithorizons-industry ft-name 0.0
EDIT 158 hithorizons-industry globaldata-name 0.875
JACCARD 158 hithorizons-industry globaldata-name 0.0
EDIT 159 hithorizons-industry govuk-name 0.875
JACCARD 159 hithorizons-industry govuk-name 0.0
EDIT 160 hithorizons-industry hithorizons-name 0.875
JACCARD 160 hithorizons-industry hithorizons-name 0.0014771048744460858
EDIT 161 hithorizons-industry infoclipper-name 0.875
JACCARD 161 hithorizons-industry infoclipper-name 0.0003161555485298767
EDIT 162 hithorizons-industry teamblind-name 0.875
JACCARD 162 hithorizons-industry teamblind-name 0.0010460251046025104
EDIT 163 hithorizons-in

Numbero di confronti:  78
EDIT 1 companiesmarketcap-share_price forbes-profit 0.8181818181818182
JACCARD 1 companiesmarketcap-share_price forbes-profit 0.00022507314877335134
EDIT 2 companiesmarketcap-share_price companiesmarketcap-pricecap 0.8181818181818182
JACCARD 2 companiesmarketcap-share_price companiesmarketcap-pricecap 0.00019948134849391582
EDIT 3 companiesmarketcap-share_price disfold-market_cap 0.8181818181818182
JACCARD 3 companiesmarketcap-share_price disfold-market_cap 0.00021781746896101068
EDIT 4 companiesmarketcap-share_price globaldata-market_cap 0.8181818181818182
JACCARD 4 companiesmarketcap-share_price globaldata-market_cap 0.0
EDIT 5 companiesmarketcap-share_price valuetoday-annual_net_income_usd 0.8095238095238095
JACCARD 5 companiesmarketcap-share_price valuetoday-annual_net_income_usd 0.00019033117624666921
EDIT 6 companiesmarketcap-share_price companiesmarketcap-revenue 0.7272727272727273
JACCARD 6 companiesmarketcap-share_price companiesmarketcap-revenue 0.00

Numbero di confronti:  120
EDIT 1 ft-revenue_2020_eu companiesmarketcap-share_price 0.8
JACCARD 1 ft-revenue_2020_eu companiesmarketcap-share_price 0.0
EDIT 2 ft-revenue_2020_eu forbes-profit 1.0
JACCARD 2 ft-revenue_2020_eu forbes-profit 0.0022187004754358162
EDIT 3 ft-revenue_2020_eu companiesmarketcap-pricecap 0.9333333333333333
JACCARD 3 ft-revenue_2020_eu companiesmarketcap-pricecap 0.03868633453938213
EDIT 4 ft-revenue_2020_eu disfold-market_cap 0.9333333333333333
JACCARD 4 ft-revenue_2020_eu disfold-market_cap 0.0
EDIT 5 ft-revenue_2020_eu globaldata-market_cap 0.9333333333333333
JACCARD 5 ft-revenue_2020_eu globaldata-market_cap 0.0016869095816464238
EDIT 6 ft-revenue_2020_eu valuetoday-annual_net_income_usd 0.8095238095238095
JACCARD 6 ft-revenue_2020_eu valuetoday-annual_net_income_usd 0.0
EDIT 7 ft-revenue_2020_eu ft-revenue_2017_eu 0.13333333333333333
EDIT 8 ft-revenue_2020_eu companiesmarketcap-revenue 0.5333333333333333
JACCARD 8 ft-revenue_2020_eu companiesmarketcap-reve

JACCARD 77 valuetoday-annual_net_income_usd companiesmarketcap-revenue 0.00045495905368516835
EDIT 78 valuetoday-annual_net_income_usd disfold-revenue 0.8571428571428571
JACCARD 78 valuetoday-annual_net_income_usd disfold-revenue 0.0003619254433586681
EDIT 79 valuetoday-annual_net_income_usd globaldata-revenue 0.8571428571428571
JACCARD 79 valuetoday-annual_net_income_usd globaldata-revenue 0.0
EDIT 80 valuetoday-annual_net_income_usd valuetoday-revenue 0.8571428571428571
JACCARD 80 valuetoday-annual_net_income_usd valuetoday-revenue 0.00045495905368516835
EDIT 81 valuetoday-annual_net_income_usd forbes-sales 0.8095238095238095
JACCARD 81 valuetoday-annual_net_income_usd forbes-sales 0.0
EDIT 82 valuetoday-annual_net_income_usd companiesmarketcap-earnings 0.7619047619047619
JACCARD 82 valuetoday-annual_net_income_usd companiesmarketcap-earnings 0.00045495905368516835
EDIT 83 valuetoday-annual_net_income_usd forbes-market_value 0.7619047619047619
JACCARD 83 valuetoday-annual_net_income_

### TASK 4

In [ ]:
print("\nMake schema\n")
remove_and_make_dir(final_schema_schema)
make_schema_final(final_datasets_base_path, final_schema_final_dictionary, final_schema_schema, ".json")